In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/PC1.arff')
df = pd.DataFrame(data[0])

In [6]:
df.shape

(705, 38)

In [7]:
df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,1.0,3.0,2.0,0.0,0.0,4.0,2.0,0.40,2.0,2.0,...,8.0,0.29,9.0,12.0,6.0,10.0,7.0,0.00,5.0,b'N'
1,2.0,5.0,2.0,1.0,3.0,8.0,3.0,0.21,4.0,2.0,...,11.0,0.15,20.0,26.0,12.0,15.0,20.0,23.53,14.0,b'Y'
2,0.0,3.0,1.0,0.0,0.0,4.0,2.0,0.14,2.0,2.0,...,5.0,0.13,27.0,31.0,11.0,10.0,15.0,0.00,14.0,b'N'
3,2.0,3.0,2.0,0.0,0.0,4.0,2.0,0.10,2.0,2.0,...,7.0,0.09,24.0,29.0,17.0,11.0,23.0,0.00,20.0,b'N'
4,19.0,17.0,13.0,0.0,0.0,4.0,15.0,0.27,2.0,2.0,...,22.0,0.20,59.0,100.0,39.0,26.0,75.0,0.00,55.0,b'N'


In [8]:
buggy = b'Y'
clean = b'N'
x = df.drop(['Defective'],axis = 'columns')

for i in range(705):
    if df.iloc[i,37] == buggy:
        df.iloc[i,37] = 1
    else:
        df.iloc[i,37] = 0

y = df['Defective']
y = y.astype(int)
y.head()

0    0
1    1
2    0
3    0
4    0
Name: Defective, dtype: int32

In [9]:
buggy = 1
clean = 0

In [10]:
from sklearn.cluster import AffinityPropagation

af_model = AffinityPropagation(damping=0.9)
af_model.fit(x)
af_prediction = af_model.predict(x)

af_prediction

array([27, 20, 20, 27, 14, 27, 31, 31, 27, 31, 14, 27, 18, 27, 27,  0, 27,
       27, 27, 27, 20, 29, 31, 27,  1, 14, 29, 29, 32, 15, 15, 31, 31, 27,
       14, 27, 27, 27, 29,  2, 33, 31,  5, 33,  5,  3, 20, 31, 20, 20, 27,
       20, 15, 31, 31,  5, 27, 31, 27, 21, 23, 20, 29, 20, 31, 18, 21, 29,
       20, 29, 27, 27, 15, 20, 31, 20, 29, 31, 32, 31, 23, 20, 29, 27, 23,
       33, 20, 27,  8, 21, 23, 23, 31, 21, 15, 20, 20, 29,  4, 20, 31, 20,
       29, 15, 14, 20, 32, 27, 27, 20,  5,  6,  7, 27, 15, 20, 10, 20, 27,
       15, 26, 14,  0,  8, 31, 15,  1, 26, 27, 33, 14, 29, 27,  9, 27, 20,
       29, 20, 27, 20, 20,  7, 20, 14, 31, 27, 27, 29, 14, 14,  8, 15, 23,
       20, 31, 27, 31, 29, 27, 32, 20, 20, 15, 20, 15, 27, 27, 15, 20, 33,
       20, 27, 15, 31, 20, 29, 29, 33, 23, 15, 24, 21, 31, 14, 20, 20, 21,
       10, 27, 11, 20, 20, 29, 20, 23, 29, 31, 20, 27, 33, 27, 32, 23, 31,
       27, 23,  6, 27, 27, 26, 15,  8, 29,  0, 27,  7, 20, 20, 31, 31, 27,
       29, 31, 20, 27, 31

In [11]:
clusters = unique(af_prediction)
print(clusters)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33]


In [12]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if af_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if af_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


In [13]:
TP = 1
TN = 1
FP = 1
FN = 1

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == af_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient
MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.07430942640434511
Accuracy : 0.9026798307475318
F-Score : 0.08


In [14]:
from sklearn.cluster import OPTICS

optics_model = OPTICS()
optics_prediction = optics_model.fit_predict(x)

optics_prediction[:5]

array([ 0, 11, 12, -1, 31])

In [15]:
clusters = unique(optics_prediction)
print(clusters)

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51]


In [16]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if optics_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if optics_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [17]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == optics_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.0766375986247729
Accuracy : 0.5092198581560283
F-Score : 0.1839622641509434


In [18]:
from sklearn.cluster import MeanShift

MeanShift_model = MeanShift()
MeanShift_prediction = MeanShift_model.fit_predict(x)

clusters = unique(MeanShift_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if MeanShift_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if MeanShift_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 1
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == MeanShift_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.17660552104615565
Accuracy : 0.9150141643059491
F-Score : 0.0909090909090909


In [19]:
from sklearn.cluster import DBSCAN

DBSCAN_model = DBSCAN()
DBSCAN_prediction = DBSCAN_model.fit_predict(x)

clusters = unique(DBSCAN_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if DBSCAN_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if DBSCAN_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == DBSCAN_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1

TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.011582185110344572
Accuracy : 0.08781869688385269
F-Score : 0.15926892950391644


In [20]:
from pyclustering.cluster.xmeans import xmeans

xmeans_instance = xmeans(x)
xmeans_instance.process()
groups = xmeans_instance.get_clusters()

xmeans_prediction = [0]*len(y)

for i in range(len(groups)):
    for u in groups[i]:
        xmeans_prediction[u] = i

clusters = unique(xmeans_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if xmeans_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if xmeans_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == xmeans_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.05488699138996578
Accuracy : 0.9063829787234042
F-Score : 0.05714285714285715
